In [1]:
# pdf-correlator by Tillman Jex
# github.com/tjex
# tillmanjex@mailbox.org

import os, glob, re, io, random, gensim, smart_open, logging, collections
import numpy as np
import pandas as pd

from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text as pdfminer_extraction
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import word_tokenize

pdfReaders = []
pdfFiles = []
docLabels = []

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored"
txtExtractDir = rootDir + '/txt-extractions'
modelDataDir = rootDir + '/model-data'
testsDir = '/Users/tillman/t-root/dev/projects/2022/pdf-correlator/tests'
zoteroDir = '/Users/tillman/t-root/zotero/storage'


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
###### PART 1 - READ, EXTRACT, TRAIN AND ASSESS ######

In [2]:
# read files
print("reading pdfs in" + str(rootDir) + " (including subdirectories)")
def read_files():
    os.chdir(rootDir)
    for file in glob.glob("**/*.pdf", recursive=True):
        try:
            pdfFiles.append(file)
            pdfReaders.append(PdfReader(file))
        except:
            print(bcolors.FAIL + "error: " + file + " is unreadable by glob.glob. Skipping file" + bcolors.ENDC)
    print(bcolors.OKGREEN + "pdf files read" + bcolors.ENDC)
    print()
read_files()

reading pdfs in/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored (including subdirectories)
error: pdf-tests/subdirectory/Simeone et al_2018_Arts and design as translational mechanisms for academic entrepreneurship.pdf is unreadable by glob.glob. Skipping file
pdf files read



In [3]:
# extract text from pdfs to designated directory and save as txt files.
def extract_to_txt():
    print("Extracting pdfs to text files (duplicate pdfs are handled automagically)")
    os.chdir(txtExtractDir)
    counter = 0
    text = ""
    for i in pdfReaders:
        counter += 1
        with open(str([i.metadata.title]) + ".txt", 'w', encoding="utf-8") as file:
      
            # add doc title to array for reference / tagging
            docLabels.append(i.metadata.title)
            try:
                for j in range(len(i.pages)):
                    # format txt file so that each document is one one line (doc2vec requirement)
                    text += i.getPage(j).extract_text()
                    text = "".join(line.strip("\n") for line in text)  

                
                    
            except Exception as exc:
                print(bcolors.FAIL + "error: " + "\"" + str(i.metadata.title) + "\"" + " not extractable with PyPDF2, trying with pdfminer" + bcolors.ENDC)
                print()
                # format txt file so that each document is one one line (doc2vec requirement)
                text += pdfminer_extraction(rootDir + "/" + pdfFiles[counter])
                text = "".join(line.strip("\n") for line in text)     
                
 
            file.write(text)
    print(bcolors.OKGREEN + "pdf extraction complete" + bcolors.ENDC)
extract_to_txt()


Extracting pdfs to text files (duplicate pdfs are handled automagically)
error: "None" not extractable with PyPDF2, trying with pdfminer

pdf extraction complete


In [4]:
# generate a training corpus from all txt files found in designated directory
class CorpusGen(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self, tokens_only=False):
        counter = 0
        for fname in os.listdir(self.dirname):
            
            with smart_open.open(fname, encoding="iso-8859-1") as f:
                for i, line in enumerate(f):
                    tokens = gensim.utils.simple_preprocess(line, min_len=3, max_len=20, deacc=True)
                    if tokens_only:
                        yield tokens
                    else:
                        yield gensim.models.doc2vec.TaggedDocument(tokens, [counter])
            counter += 1
        
trainCorpus = list(CorpusGen('/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions'))

In [5]:
# save the entire corpus to a txt file
with open(modelDataDir + "/train-corpus.txt", 'w') as file:
    file.write(str(trainCorpus))

In [6]:
# establish a model and build the vocab
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(trainCorpus)
model.train(trainCorpus, total_examples=model.corpus_count, epochs=model.epochs)

2022-09-30 10:33:44,637 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>', 'datetime': '2022-09-30T10:33:44.637837', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'created'}
2022-09-30 10:33:44,640 : INFO : collecting all words and their counts
2022-09-30 10:33:44,641 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-09-30 10:33:44,723 : INFO : collected 13122 word types and 12 unique tags from a corpus of 12 examples and 669596 words
2022-09-30 10:33:44,723 : INFO : Creating a fresh vocabulary
2022-09-30 10:33:44,744 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 12698 unique words (96.77% of original 13122, drops 424)', 'datetime': '2022-09-30T10:33:44.744148', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1

In [7]:
# generate and format data files for tensorboard visualisation
os.chdir(modelDataDir)
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)
%run ../scripts/word2vec2tensor.py -i doc_tensor.w2v -o pdf_plot

text = ""    
with open('pdf_plot_metadata.tsv', 'w') as file:
    file.write('title\n')
    for fname in os.listdir(txtExtractDir):
        if fname.endswith('.txt'):
            text = fname
            text = re.sub('\[\'', '', text)
            text = re.sub('\'\].txt', '', text)
            text = re.sub('\[', '', text)
            text = re.sub('\].txt', '', text)     
            print(text)
            file.write("%s\n" % text)
        else:
            continue
        

2022-09-30 10:34:04,295 : WARNING : attribute doc_count not present in KeyedVectors<vector_size=50, 12 keys>; will store in internal index_to_key order
2022-09-30 10:34:04,298 : INFO : storing 12x50 projection weights into doc_tensor.w2v
2022-09-30 10:34:04,306 : INFO : running ../scripts/word2vec2tensor.py -i doc_tensor.w2v -o pdf_plot
2022-09-30 10:34:04,308 : INFO : loading projection weights from doc_tensor.w2v
2022-09-30 10:34:04,310 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (12, 50) matrix of type float32 from doc_tensor.w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-09-30T10:34:04.310748', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'load_word2vec_format'}
2022-09-30 10:34:04,312 : INFO : 2D tensor file saved to pdf_plot_tensor.tsv
2022-09-30 10:34:04,313 : INFO : Tensor metadata file saved to pdf_plot_metadata.tsv
2022-09-30 10:34:0

IAFOR Journal of Cultural Studies – Volume 6 – Issue 1 
frvir-2022-779148 1..5
ShareVR: Enabling Co-Located Experiences for Virtual Reality between HMD and Non-HMD Users
The effects of visual context and individual differences on perception and evaluation of modern art and graffiti art
User attention and behaviour in virtual reality art encounter
Microsoft Word - 48710116.DOC
pone.0099019 1..8
Making Art Therapy Virtual: Integrating Virtual Reality Into Art Therapy With Adolescents
Microsoft Word - CHI2018_LucidDreaming_v5.docx
g5grap.lo
None


In [8]:
# word occurence check
checkWord = "internet"
print("\"" + str(checkWord) + "\"" + " appears this many times in corpus:") 
print({model.wv.get_vecattr(checkWord, 'count')})
print()

"internet" appears this many times in corpus:
{96}



In [9]:
# assessing the model
print("assessing the model (this can take a while)")
ranks = []
secondRanks = []
for doc_id in range(len(trainCorpus)):
        inferredVector = model.infer_vector(trainCorpus[doc_id].words)
        sims = model.dv.most_similar([inferredVector], topn=len(model.dv))
        rank = [docid for docid, sim in sims].index(doc_id)
        ranks.append(rank)
        secondRanks.append(sims[1])

counter = collections.Counter(ranks)
print(bcolors.OKGREEN + "model assessed" + bcolors.ENDC)
print()
print(bcolors.OKBLUE + "model trained and assessed successfully" + bcolors.ENDC)

assessing the model (this can take a while)
model assessed

model trained and assessed successfully


In [ ]:
###### PART 2 - CHECK SIMILARITY BETWEEN CORPUS AND INCOMING DOCUMENT ######

In [13]:
# import new document

print('importing latin pdf')
with smart_open.open(testsDir + '/similarity-test.txt', 'w') as test:
    text = pdfminer_extraction(testsDir + '/german.pdf')
    text = "".join(line.strip("\n") for line in text) 
    test.write(text)
    
print()

importing latin pdf



In [14]:
# tokenize and tag new document
print('tag and tokenize pdf for doc2vec processing')
def read_text(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
                
similarityTest = list(read_text(testsDir + '/similarity-test.txt'))

print()

tag and tokenize pdf for doc2vec processing



In [15]:
# check for similarity against the entire corpus
print('compare similarity between incoming pdf and text corpus (the model)')
novel_vector = model.infer_vector(similarityTest[0].words)
similarity_scores = model.dv.most_similar([novel_vector])
average = 0
for score in similarity_scores:
    average += score[1]
overall_similarity = average/len(similarity_scores)

print("> Incoming pdf is " + format((overall_similarity - 1) * -1, '.0%') + " similar to corpus")


compare similarity between incoming pdf and text corpus (the model)
> Incoming pdf is 38% similar to corpus
